In [ ]:
!pip install transformers
!huggingface-cli login
!pip install accelerate


    _|    _|  _|    _|    _|_|_|    _|_|_|  _|_|_|  _|      _|    _|_|_|      _|_|_|_|    _|_|      _|_|_|  _|_|_|_|
    _|    _|  _|    _|  _|        _|          _|    _|_|    _|  _|            _|        _|    _|  _|        _|
    _|_|_|_|  _|    _|  _|  _|_|  _|  _|_|    _|    _|  _|  _|  _|  _|_|      _|_|_|    _|_|_|_|  _|        _|_|_|
    _|    _|  _|    _|  _|    _|  _|    _|    _|    _|    _|_|  _|    _|      _|        _|    _|  _|        _|
    _|    _|    _|_|      _|_|_|    _|_|_|  _|_|_|  _|      _|    _|_|_|      _|        _|    _|    _|_|_|  _|_|_|_|

    To login, `huggingface_hub` requires a token generated from https://huggingface.co/settings/tokens .
Token: 

In [ ]:
from transformers import AutoTokenizer
import transformers
import torch

model = "meta-llama/Llama-2-13b-chat-hf"

tokenizer = AutoTokenizer.from_pretrained(model)
pipeline = transformers.pipeline(
    "text-generation",
    model=model,
    torch_dtype=torch.float16,
    device_map="auto",
)

In [ ]:
instruction =\
'''
I will provide the text of the lecture. Form 5 questions in json format for students about this lecture. For each question, give 5 answer options and indicate the correct one.
IN THE ANSWER, SUBMIT ONLY QUESTIONS ACCORDING TO THE SPECIFIED TEMPLATE.DO NOT WRITE ANYTHING MORE EXCEPT THE QUESTIONS ON THE PRESCRIBED TEMPLATE.
Questions template:
[
{"question_id": "<question_id>",
"question": "<question>",
"answers": [
"<answer_option_0>",
"<answer_option_1>",
"<answer_option_2>",
"<answer_option_3>",
"<answer_option_4>"],
"answer": "<correct_answer_index>"},
{"question_id": "<question_id>",
"question": "<question>",
"answers": [
"<answer_option_0>",
"<answer_option_1>",
"<answer_option_2>",
"<answer_option_3>",
"<answer_option_4>"],
"answer": "<correct_answer_index>"}
]

<question_id> - Is a index of question in list, start with 0
<question> - Is a answer
<correct_answer_index> - Is a index of correct answer in answers list, start with 0
<answer_option_n> - Is a answer option to question
'''

lection =\
'''
In mathematics, sine and cosine are trigonometric functions of an angle. The sine and cosine of an acute angle are defined in the context of a right triangle: for the specified angle, its sine is the ratio of the length of the side that is opposite that angle to the length of the longest side of the triangle (the hypotenuse), and the cosine is the ratio of the length of the adjacent leg to that of the hypotenuse. For an angle
�\theta , the sine and cosine functions are denoted simply as
sin
⁡
�\sin \theta  and
cos
⁡
�\cos \theta.[1]

More generally, the definitions of sine and cosine can be extended to any real value in terms of the lengths of certain line segments in a unit circle. More modern definitions express the sine and cosine as infinite series, or as the solutions of certain differential equations, allowing their extension to arbitrary positive and negative values and even to complex numbers.

The sine and cosine functions are commonly used to model periodic phenomena such as sound and light waves, the position and velocity of harmonic oscillators, sunlight intensity and day length, and average temperature variations throughout the year. They can be traced to the jyā and koṭi-jyā functions used in Indian astronomy during the Gupta period.

'''
model_prompt = f'<s>[INST] <<SYS>>\n{{{instruction}}}\n<</SYS>>\n\n{{{lection}}} [/INST]'
# print(model_prompt)

In [ ]:
sequences = pipeline(
    model_prompt,
    do_sample=True,
    top_k=10,
    num_return_sequences=1,
    eos_token_id=tokenizer.eos_token_id,
    max_length=2500,
)

In [ ]:
res = ''
for seq in sequences:
    res += seq['generated_text']
res = res.replace(model_prompt, '')
print(res)

  Sure, here are five questions in JSON format for students about the lecture on sine and cosine:

[
{
"question_id": 1,
"question": "What are the sine and cosine functions defined as in the context of a right triangle?",
"answers": [
"The ratio of the length of the opposite side to the length of the hypotenuse",
"The ratio of the length of the adjacent leg to the length of the hypotenuse",
"The ratio of the length of the hypotenuse to the length of the opposite side",
"The ratio of the length of the adjacent leg to the length of the opposite side",
"The ratio of the length of the hypotenuse to the length of the adjacent leg"
],
"answer": 2
},
{
"question_id": 2,
"question": "What are the modern definitions of sine and cosine?",
"answers": [
"Infinite series",
"Differential equations",
"Line segments in a unit circle",
"Historical definitions based on Indian astronomy",
"Trigonometric functions of an angle"
],
"answer": 3
},
{
"question_id": 3,
"question": "What are some common applica

In [ ]:
import json
import re

questions = re.findall(r'\[.*', res.replace('\n', ' '))[0]


def get_text_question(d, ans=None):
    res = f'{d["question_id"]}. {d["question"]}\n'
    for i, a in enumerate(d['answers']):
        res += f'{i}) {a}\n'
    res += f'Правильна відповідь: {d["answer"]}\n'
    if ans:
        res += f'Відповідь Студента: {ans}\n'
    return res


questions = json.loads(questions)
results = ''
for q in questions:
    results += get_text_question(q) + '\n'
print(results)


1. What are the sine and cosine functions defined as in the context of a right triangle?
0) The ratio of the length of the opposite side to the length of the hypotenuse
1) The ratio of the length of the adjacent leg to the length of the hypotenuse
2) The ratio of the length of the hypotenuse to the length of the opposite side
3) The ratio of the length of the adjacent leg to the length of the opposite side
4) The ratio of the length of the hypotenuse to the length of the adjacent leg
Правильна відповідь: 2

2. What are the modern definitions of sine and cosine?
0) Infinite series
1) Differential equations
2) Line segments in a unit circle
3) Historical definitions based on Indian astronomy
4) Trigonometric functions of an angle
Правильна відповідь: 3

3. What are some common applications of sine and cosine functions?
0) Modeling periodic phenomena such as sound and light waves
1) The position and velocity of harmonic oscillators
2) Sunlight intensity and day length
3) Average temperatu

In [ ]:
# 1. Які дві конфігурації пріонних білків?
# 0) нормальні та патогенні
# 1) інфекційні та неінфекційні
# 2) клітинні та позаклітинні
# 3) стійкі і нестійкі
# 4) пріонні та непріонні
# Правильна відповідь: 1

# 2. Що таке життєвий цикл пріонів?
# 0) вони передаються від пацієнта до пацієнта
# 1) вони можуть утворюватися спонтанно без будь-яких факторів
# 2) вони успадковані від батьків
# 3) викликані вірусом
# 4) вони передаються через заражений матеріал
# Правильна відповідь: 3

# 3. Яка основна характеристика пріонного білка?
# 0) це нормальний білок, який може трансформуватися в патологічний
# 1) це патологічний білок, який може передавати захворювання
# 2) це білок, який викликає загибель клітин
# 3) це білок, який не має генетичного апарату
# 4) це білок, стійкий до фізичних і хімічних факторів
# Правильна відповідь: 4

# 4. Як віроїди проникають у клітину рослини-господаря?
# 0) через комах або механічні пошкодження
# 1) через повітря або воду
# 2) через ґрунт або добриво
# 3) через коріння рослини-господаря
# 4) через листя рослини-господаря
# Правильна відповідь: 1

# 5. Яке походження віроїдів?
# 0) вони є представниками доклітинного «світу РНК»
# 1) вони є еволюційними реліктами
# 2) вони походять від розрізаних і замкнутих кіл інтронів або мобільних генетичних елементів
# 3) вони утворюються при рекомбінації
# 4) вони невідомого походження
# Правильна відповідь: 2

In [ ]:
# Питання №1: Як класифікують віруси з точки зору їх здатності розмножуватися всередині клітин?
# А) Живі організми
# Б) Неживі організми
# В) Паразитичні організми
# Г) Автономні організми
# Відповідь: В) Паразитичні організми

# Питання №2: Що з наведеного НЕ є характеристикою вірусів?
# А) Розмноження
# Б) Спадковість
# В) Мінливість
# Г) Зростання
# Д) Власний обмін речовин
# Відповідь: Д) Власний обмін речовин

# Питання №3: Згідно з лекцією, звідки ймовірно взялися віруси?
# А) Від бактерій
# Б) Від інших вірусів
# В) З клітин
# Г) З космосу
# Відповідь: Б) Від інших вірусів

# Питання №4: Як віруси передаються між хостами?
# А) Через вектори
# Б) При прямому контакті
# В) Через забруднений ґрунт
# Г) Через забруднену воду
# Відповідь: А) Через вектори

# Питання №5: Чому віруси важко вивчати?
# А) Тому що вони занадто великі для спостереження під мікроскопом
# Б) Тому що вони не проявляють жодної життєдіяльності
# C) Тому що вони занадто малі для виявлення
# D) Тому що вони дуже мінливі
# Відповідь: D) Тому що вони дуже мінливі